# State Management
State in LangGraph is a typed dictionary that flows through the graph. Nodes read inputs from the state and return updates that merge into the state. This enables clear data dependencies, parallel execution (fan-out), and synchronized aggregation (fan-in) without explicit message passing.

### Use Case: Automobile Industry – Intelligent Maintenance Advisor
The workflow processes multiple sources in parallel:
- Vehicle diagnostics (OBD-II logs) → fault analysis
- Customer feedback → issues summary
- Maintenance manuals lookup → recommended procedures

A final aggregator node merges the three results into a unified diagnostic summary using an OpenAI LLM.


In [1]:
from typing import TypedDict

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END


In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [3]:
class MaintenanceState(TypedDict):
    obd_logs: str
    customer_feedback: str
    manuals_query: str
    sensor_analysis: str
    feedback_summary: str
    recommended_solution: str
    diagnostic_summary: str


#### Define Processing Nodes (Channels)

In [4]:
def analyze_sensors(state: MaintenanceState):
    prompt = f"""
    Analyze these vehicle sensor readings and identify issues:
    
    Sensor Data:
    {state['obd_logs']}
    """
    response = llm.invoke(prompt)
    analysis = response.content.strip()
    return {"sensor_analysis": analysis}


#### Node 2 — Maintenance Prediction Node

In [5]:
def analyze_feedback(state: MaintenanceState):
    prompt = f"""
    Analyze this customer feedback and extract key issues:
    
    Customer Feedback:
    {state['customer_feedback']}
    """
    response = llm.invoke(prompt)
    analysis = response.content.strip()
    return {"feedback_summary": analysis}


#### Node 3 — Technician Report Generation Node

In [6]:
def lookup_manual(state: MaintenanceState):
    prompt = f"""
    Search maintenance database for solutions based on:
    Sensor Data: {state['obd_logs'][:100]}...
    Customer Feedback: {state['customer_feedback'][:100]}...
    Query: {state['manuals_query']}
    
    Provide relevant maintenance procedures and solutions.
    """
    response = llm.invoke(prompt)
    lookup = response.content.strip()
    return {"recommended_solution": lookup}


#### Node 4 — Final Output Node

In [7]:
def generate_final_report(state: MaintenanceState):
    sensor_analysis = state.get("sensor_analysis", "")
    feedback_summary = state.get("feedback_summary", "")
    recommended_solution = state.get("recommended_solution", "")
    
    prompt = f"""
    Create a comprehensive diagnostic report combining:
    
    Sensor Analysis: {sensor_analysis}
    Customer Feedback: {feedback_summary}
    Manual Solutions: {recommended_solution}
    
    Provide a unified maintenance recommendation.
    """
    response = llm.invoke(prompt)
    final_report = response.content.strip()
    return {"diagnostic_summary": final_report}


#### Build and Connect the Workflow Graph

In [8]:
# Build state-managed graph
graph = StateGraph(MaintenanceState)

# Worker nodes
graph.add_node("analyze_sensors", analyze_sensors)
graph.add_node("analyze_feedback", analyze_feedback)
graph.add_node("lookup_manual", lookup_manual)

# Aggregator
graph.add_node("aggregate", generate_final_report)

# Fan-out from START
graph.add_edge(START, "analyze_sensors")
graph.add_edge(START, "analyze_feedback")
graph.add_edge(START, "lookup_manual")

# Fan-in to aggregator
graph.add_edge("analyze_sensors", "aggregate")
graph.add_edge("analyze_feedback", "aggregate")
graph.add_edge("lookup_manual", "aggregate")

# Finish at END via aggregate
graph.add_edge("aggregate", END)


In [9]:
# Example input
obd_logs = """
Engine Temperature: 105°C
Battery Voltage: 11.2V
Tire Pressure: Low
Oil Pressure: 18 PSI
"""

customer_feedback = """
Customer complains about:
- Engine overheating
- Poor fuel efficiency
- Strange noises when braking
"""

manuals_query = "P0128 overheating; low oil pressure"

# Initialize state
state: MaintenanceState = {
    "obd_logs": obd_logs,
    "customer_feedback": customer_feedback,
    "manuals_query": manuals_query,
    "sensor_analysis": "",
    "feedback_summary": "",
    "recommended_solution": "",
    "diagnostic_summary": "",
}

# Run graph
app = graph.compile()
result_state = app.invoke(state)


#### Review Channel Data

In [10]:
# Review results from state
print("🔧 Sensor Analysis:\n", result_state.get("sensor_analysis"), "\n")
print("💬 Feedback Summary:\n", result_state.get("feedback_summary"), "\n")
print("📚 Recommended Solution:\n", result_state.get("recommended_solution"), "\n")
print("📋 Diagnostic Summary:\n", result_state.get("diagnostic_summary"), "\n")


🔧 Sensor Analysis:
 Based on the provided vehicle sensor readings, here are the analyses and potential issues identified:

1. **Engine Temperature: 105°C**
   - **Analysis**: This temperature is higher than the typical operating range for most engines, which is usually between 90°C and 100°C. An engine temperature of 105°C could indicate overheating.
   - **Potential Issues**: Possible causes for overheating include low coolant levels, a malfunctioning thermostat, a failing water pump, or a blocked radiator. Immediate attention is recommended to prevent engine damage.

2. **Battery Voltage: 11.2V**
   - **Analysis**: A healthy car battery typically reads between 12.4V and 12.7V when the engine is off and around 13.7V to 14.7V when the engine is running (due to the alternator charging the battery). A reading of 11.2V suggests that the battery is undercharged or failing.
   - **Potential Issues**: This could indicate a weak battery, a failing alternator, or an electrical drain on the bat